## Julia on Colab Instructions
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. If you need a GPU: _Runtime_ > _Change runtime type_ > _Harware accelerator_ = _GPU_.
3. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia and other packages (if needed, update `JULIA_VERSION` and the other parameters). This takes a couple of minutes.
4. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the next section.

_Notes_:
* If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2, 3 and 4.
* After installation, if you want to change the Julia version or activate/deactivate the GPU, you will need to reset the Runtime: _Runtime_ > _Factory reset runtime_ and repeat steps 3 and 4.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.1" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia BenchmarkTools Flux https://github.com/Dale-Black/DistanceTransforms.jl.git"
JULIA_PACKAGES_IF_GPU="CUDA" # or CuArrays for older Julia versions
JULIA_NUM_THREADS=1
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Success! Please reload this page and jump to the next section."
fi

Installing Julia 1.6.1 on the current Colab Runtime...
2021-06-23 20:03:29 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.1-linux-x86_64.tar.gz [112784227/112784227] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package BenchmarkTools...
Installing Julia package Flux...
Installing Julia package https://github.com/Dale-Black/DistanceTransforms.jl.git...


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system:

In [1]:
versioninfo()

Julia Version 1.6.1
Commit 6aaedecc44 (2021-04-23 05:59 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 1


## Import Packages

In [2]:
using BenchmarkTools
using DistanceTransforms
using CUDA
using Flux

## Load test data

In [13]:
x = rand([0,1], (112, 112, 96, 2, 1))
y = rand([0,1], (112, 112, 96, 2, 1))

x_dtm = DistanceTransforms.compute_dtm(x)
y_dtm = DistanceTransforms.compute_dtm(y)

x, y = x |> gpu, y |> gpu
x_dtm, y_dtm = x_dtm |> gpu, y_dtm |> gpu;

## Vanilla dice loss

In [14]:
@benchmark DistanceTransforms.dice_loss($x, $y)

BenchmarkTools.Trial: 
  memory estimate:  57.38 KiB
  allocs estimate:  2982
  --------------
  minimum time:     2.016 ms (0.00% GC)
  median time:      2.039 ms (0.00% GC)
  mean time:        2.094 ms (1.27% GC)
  maximum time:     17.742 ms (45.60% GC)
  --------------
  samples:          2379
  evals/sample:     1

## Vanilla Hausdorff loss

In [15]:
@benchmark DistanceTransforms.hd_loss($x, $y, $x_dtm, $y_dtm)

BenchmarkTools.Trial: 
  memory estimate:  27.55 KiB
  allocs estimate:  897
  --------------
  minimum time:     856.563 μs (0.00% GC)
  median time:      866.499 μs (0.00% GC)
  mean time:        884.438 μs (0.75% GC)
  maximum time:     26.982 ms (46.85% GC)
  --------------
  samples:          5600
  evals/sample:     1

In [16]:
#= Need to fix scalar indexing on GPU
@benchmark DistanceTransforms.hd_loss($x, $y, $DistanceTransforms.compute_dtm(x), $DistanceTransforms.compute_dtm(y))
=#

## Parallelized dice loss

In [17]:
#= Need to fix scalar indexing on GPU
@benchmark DistanceTransforms.dice_lossP($x_small, $y_small)
=#

## Parallelized Hausdorff loss

In [18]:
#= Need to fix scalar indexing on GPU
@benchmark DistanceTransforms.hd_lossP($x, $y, $x_dtm, $y_dtm)
=#

In [19]:
#= Need to fix scalar indexing on GPU
@benchmark DistanceTransforms.hd_lossP($x, $y, $DistanceTransforms.compute_dtm(x), $DistanceTransforms.compute_dtm(y))
=#